В этом ноутбуке я сделал все шаги пайплайна в ручную что бы узнать какие есть ошибки и оценить модель в целом

In [1]:
import dill
import pandas as pd
import hashlib
import math

from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('final_data', low_memory=False)
df = pd.DataFrame(df)

Работа с колонками

In [6]:
def process_screen_resolution(df, resolution_column='device_screen_resolution'):
    # Делим разрешение экрана на x и y
    df[['x_pixel', 'y_pixel']] = df[resolution_column].str.split('x', expand=True)
    df['x_pixel'] = pd.to_numeric(df['x_pixel'])
    df['y_pixel'] = pd.to_numeric(df['y_pixel'])

    # Логарифмируем значения
    df['log_pixel'] = df['x_pixel'] * df['y_pixel']
    df['log_pixel'] = df['log_pixel'].apply(lambda x: math.log(x))
    df['x_pixel'] = df['x_pixel'].apply(lambda x: min(x, 2500))
    df['y_pixel'] = df['y_pixel'].apply(lambda x: min(x, 2000))

    # Обработка выбросов
    mx = df['x_pixel'].median()
    my = df['y_pixel'].median()
    zero_pixel_rows = df['log_pixel'] == 0
    df.loc[zero_pixel_rows, 'log_pixel'] = math.log(mx * my)
    df.loc[zero_pixel_rows, 'x_pixel'] = mx
    df.loc[zero_pixel_rows, 'y_pixel'] = my

    # Умножаем x на y и заменяем значения в 'device_screen_resolution'
    df[resolution_column] = df['x_pixel'].astype(str) + 'x' + df['y_pixel'].astype(str)

    return df

In [7]:
df = process_screen_resolution(df)

In [8]:
df

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target,x_pixel,y_pixel,log_pixel
0,861567602239642895.1635425553.1635425553,2.005993e+08,2021-10-28,15:52:33,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,UbQpozKfTgYxQovHjkQM,...,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow,1,412,915,12.839947
1,8703059496721564670.1630389395.1630389395,2.026339e+09,2021-08-31,08:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,pHqgLcdBQPSLcrTpLkdr,...,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Samsung Internet,Russia,Kubinka,1,360,640,12.347572
2,5366140572278903224.1635701148.1635701148,1.249402e+09,2021-10-31,20:25:48,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,...,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Veliky Novgorod,0,360,640,12.347572
3,5354870681162512304.1624802278.1624802278,1.246778e+09,2021-06-27,16:00:00,2,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,...,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Kirov,0,1920,1080,14.544797
4,325238689346041728.1638872959.1638872959,7.572553e+07,2021-12-07,13:29:19,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Saransk,0,360,720,12.465355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434912,6944657193759066052.1625213471.1625213471,1.616929e+09,2021-07-02,11:00:00,4,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,...,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Chrome,Russia,Khimki,0,390,844,12.704299
434913,998273288816685235.1622012085.1622012085,2.324286e+08,2021-05-26,09:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,...,Samsung,ZkRlaMsFhSBPslXBtfiG,412x869,Chrome,Russia,Moscow,0,412,869,12.788366
434914,5699137684613566500.1634668580.1634668678,1.326934e+09,2021-10-19,21:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,...,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Volgograd,0,393,873,12.745745
434915,8680929029191131182.1632657455.1632657455,2.021186e+09,2021-09-26,14:00:00,1,RmEBuqrriAfAVsLQQmhk,cpc,KgicpPxiEQfzPlPwQZJq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,...,Vivo,AuMdmADEIoPXiWpTsBEj,393x876,Chrome,Russia,Saint Petersburg,0,393,876,12.749176


In [25]:
# На всякий случай создаем копию и работаем с ней
df_copy = df.copy()

# Разделяем колонку device_screen_resolution на длину и ширину
df_copy['width'] = pd.to_numeric(df_copy['device_screen_resolution'].str.split('x').str[0])
df_copy['height'] = pd.to_numeric(df_copy['device_screen_resolution'].str.split('x').str[1])

# Работаем с колонками visit_date и visit_time и создаем переменные основанные на каждом из цифр(год, месяц, день, час, и минуты) 
df_copy['datetime'] = pd.to_datetime(df_copy['visit_date'], format='%Y-%m-%d') + \
                 pd.to_timedelta(df_copy['visit_time'])
df_copy['year'] = df_copy['datetime'].dt.year
df_copy['month'] = df_copy['datetime'].dt.month
df_copy['day'] = df_copy['datetime'].dt.day
df_copy['hour'] = df_copy['datetime'].dt.hour
df_copy['minute'] = df_copy['datetime'].dt.minute

# Решаю объеденить колонки Страну и Город через "/"
df_copy['geo'] = df_copy['geo_country'] + '/' + df_copy['geo_city']

# Удаляю ненужные колонки
drop_col = ['session_id', 'client_id', 'device_screen_resolution', 'visit_date', 'visit_time', 'datetime', 'geo_country', 'geo_city']

df_copy = df_copy.drop(drop_col, axis=1)

Разделяю на фичи и целевую переменную

In [26]:
X = df_copy.drop('target', axis=1)
y = df_copy['target']

Создаю листы с названиями колонок которые нужно преоброзовать

In [27]:
scal_columns = ['visit_number', 'width', 'height', 'year', 'month', 'day', 'hour', 'minute']
encod_columns = ['utm_medium', 'device_category', 'device_os', 'device_brand', 'device_browser', 'geo']
hash_columns = ['utm_source', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_model']

Колонки scal_columns преобразовываю с помощью StandardScaler

In [28]:
scaler = StandardScaler()
X[scal_columns] = scaler.fit_transform(X[scal_columns])

Колонки hash_columns решаю хешировать, потому что уникальных значении слишком много, и создав дамми я чисто физический не смогу обработать и обучить датафрейм

In [29]:
X[hash_columns] = X[hash_columns].applymap(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())

C:\Users\Акти\AppData\Local\Temp\ipykernel_13196\4261181228.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X[hash_columns] = X[hash_columns].applymap(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())


Колонки encod_columns обрабатываю с помощью OneHotEncoder

In [30]:
encoder = OneHotEncoder()
X = encoder.fit_transform(X[encod_columns])

Создаю модели машинного обучения Логистической регрессии и Случайных лесов

In [31]:
log_reg = LogisticRegression(n_jobs=-1)
rfc = RandomForestClassifier(n_jobs=-1)

In [32]:
log_reg_cvs = cross_val_score(log_reg, X, y, cv=4, scoring='roc_auc')

In [33]:
log_reg_cvs.mean()

0.6858957058069366

In [34]:
rfc_cvs = cross_val_score(rfc, X, y, cv=4, scoring='roc_auc')

In [35]:
rfc_cvs.mean()

0.748781057137502

В целом показатели не плохие, и время обучения в пределах часа